<a href="https://colab.research.google.com/github/navyajammalamadaka/Ml-for-Cybersecurity/blob/main/FinalProject-DeepID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!tar -xvf "/content/drive/MyDrive/aligned_images_DB.tar.gz"

Streaming output truncated to the last 5000 lines.
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1769.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1827.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1795.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1825.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1841.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1806.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1768.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1757.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1804.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1808.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1787.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1812.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1805.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1828.jpg
aligned_images_DB/Flavia_Delaroli/3/aligned_detect_3.1830.jpg
aligned_images_DB/F

In [5]:
#! /usr/bin/python
import os
from PIL import Image

In [6]:
def crop_img_by_half_center(src_file_path, dest_file_path):
    im = Image.open(src_file_path)
    x_size, y_size = im.size
    start_point_xy = x_size / 4
    end_point_xy   = x_size / 4 + x_size / 2
    box = (start_point_xy, start_point_xy, end_point_xy, end_point_xy)
    new_im = im.crop(box)
    new_new_im = new_im.resize((47,55))
    new_new_im.save(dest_file_path)

In [7]:
def walk_through_the_folder_for_crop(aligned_db_folder, result_folder):
    if not os.path.exists(result_folder):
        os.mkdir(result_folder)
    
    i = 0
    img_count = 0
    for people_folder in os.listdir(aligned_db_folder):
        src_people_path = aligned_db_folder + people_folder + '/'
        dest_people_path = result_folder + people_folder + '/'
        if not os.path.exists(dest_people_path):
            os.mkdir(dest_people_path)
        for video_folder in os.listdir(src_people_path):
            src_video_path = src_people_path + video_folder + '/'
            dest_video_path = dest_people_path + video_folder + '/'
            if not os.path.exists(dest_video_path):
                os.mkdir(dest_video_path)
            for img_file in os.listdir(src_video_path):
                src_img_path = src_video_path + img_file
                dest_img_path = dest_video_path + img_file
                crop_img_by_half_center(src_img_path, dest_img_path)
            i += 1
            img_count += len(os.listdir(src_video_path))

In [9]:
if __name__ == '__main__':
    aligned_db_folder = "/content/drive/MyDrive/aligned_images_DB"
    result_folder = "/content/drive/MyDrive/crop_images_DB"
    if not aligned_db_folder.endswith('/'):
        aligned_db_folder += '/'
    if not result_folder.endswith('/'):
        result_folder += '/'
    walk_through_the_folder_for_crop(aligned_db_folder, result_folder)

In [10]:
import os.path
import random

In [11]:
def fatch_pics_for_one_user(people_path):
    people_imgs = []
    for video_folder in os.listdir(people_path):
        for video_file_name in os.listdir(os.path.join(people_path, video_folder)):
            people_imgs.append(os.path.join(people_path, video_folder, video_file_name))
    random.shuffle(people_imgs)
    return people_imgs

In [12]:
def build_dataset(src_folder):
    total_people, total_picture = 0, 0
    test_people, valid_set, train_set = [], [], []
    label = 0

    for people_folder in os.listdir(src_folder):
        people_imgs = fatch_pics_for_one_user(os.path.join(src_folder, people_folder))
        total_people += 1
        total_picture += len(people_imgs)
        if len(people_imgs) < 100:
            test_people.append(people_imgs)
        else:
            valid_set += zip(people_imgs[:10], [label]*10)
            train_set += zip(people_imgs[10:100], [label]*90)
            label += 1

    test_set = []
    for i, people_imgs in enumerate(test_people):
        for k in range(5):
            same_pair = random.sample(people_imgs, 2)
            test_set.append((same_pair[0], same_pair[1], 1))
        for k in range(5):
            j = i;
            while j == i:
                j = random.randint(0, len(test_people)-1)
            test_set.append((random.choice(test_people[i]), random.choice(test_people[j]), 0))

    random.shuffle(test_set)
    random.shuffle(valid_set)
    random.shuffle(train_set)

    print('\tpeople\tpicture')
    print('total:\t%6d\t%7d' % (total_people, total_picture))
    print('test:\t%6d\t%7d' % (len(test_people), len(test_set)))
    print('valid:\t%6d\t%7d' % (label, len(valid_set)))
    print('train:\t%6d\t%7d' % (label, len(train_set)))
    return test_set, valid_set, train_set

In [13]:
def set_to_csv_file(data_set, file_name):
    with open(file_name, "w") as f:
        for item in data_set:
            print(" ".join(map(str, item)), file=f)

In [14]:
if __name__ == '__main__':
    random.seed(7)
    src_folder     = "/content/drive/MyDrive/crop_images_DB"
    test_set_file  = "/content/drive/MyDrive/test_set.csv"
    valid_set_file = "/content/drive/MyDrive/valid_set.csv"
    train_set_file = "/content/drive/MyDrive/train_set.csv"
    if not src_folder.endswith('/'):
        src_folder += '/'
    
    test_set, valid_set, train_set = build_dataset(src_folder)
    set_to_csv_file(test_set,  test_set_file)
    set_to_csv_file(valid_set, valid_set_file)
    set_to_csv_file(train_set, train_set_file)

	people	picture
total:	   468	 216760
test:	    84	    840
valid:	   384	   3840
train:	   384	  34560


In [24]:
import pickle
import numpy as np
from PIL import Image

In [25]:
def vectorize_imgs(img_path):
    with Image.open(img_path) as img:
        arr_img = np.asarray(img, dtype='float32')
        return arr_img

In [26]:
def read_csv_file(csv_file):
    x, y = [], []
    with open(csv_file, "r") as f:
        for line in f.readlines():
            path, label = line.strip().split()
            x.append(vectorize_imgs(path))
            y.append(int(label))
    return np.asarray(x, dtype='float32'), np.asarray(y, dtype='int32')

In [27]:
def read_csv_pair_file(csv_file):
    x1, x2, y = [], [], []
    with open(csv_file, "r") as f:
        for line in f.readlines():
            p1, p2, label = line.strip().split()
            x1.append(vectorize_imgs(p1))
            x2.append(vectorize_imgs(p2))
            y.append(int(label))
    return np.asarray(x1, dtype='float32'), np.asarray(x2, dtype='float32'), np.asarray(y, dtype='int32')





In [28]:
def load_data():
    with open('/content/drive/MyDrive/dataset.pkl', 'rb') as f:
        testX1 = pickle.load(f)
        testX2 = pickle.load(f)
        testY  = pickle.load(f)
        validX = pickle.load(f)
        validY = pickle.load(f)
        trainX = pickle.load(f)
        trainY = pickle.load(f)
        return testX1, testX2, testY, validX, validY, trainX, trainY

In [29]:
if __name__ == '__main__':
    testX1, testX2, testY = read_csv_pair_file('/content/drive/MyDrive/test_set.csv')
    validX, validY = read_csv_file('/content/drive/MyDrive/valid_set.csv')
    trainX, trainY = read_csv_file('/content/drive/MyDrive/train_set.csv')

    print(testX1.shape, testX2.shape, testY.shape)
    print(validX.shape, validY.shape)
    print(trainX.shape, trainY.shape)

    print(trainY)

    with open('/content/drive/MyDrive/dataset.pkl', 'wb') as f:
        pickle.dump(testX1, f, pickle.HIGHEST_PROTOCOL)
        pickle.dump(testX2, f, pickle.HIGHEST_PROTOCOL)
        pickle.dump(testY , f, pickle.HIGHEST_PROTOCOL)
        pickle.dump(validX, f, pickle.HIGHEST_PROTOCOL)
        pickle.dump(validY, f, pickle.HIGHEST_PROTOCOL)
        pickle.dump(trainX, f, pickle.HIGHEST_PROTOCOL)
        pickle.dump(trainY, f, pickle.HIGHEST_PROTOCOL)

(840, 55, 47, 3) (840, 55, 47, 3) (840,)
(3840, 55, 47, 3) (3840,)
(34560, 55, 47, 3) (34560,)
[184 236 165 ... 270   4 128]


In [57]:
import numpy as np
import tensorflow as tf

In [58]:

def load_data():
    with open('/content/drive/MyDrive/dataset.pkl', 'rb') as f:
        testX1 = pickle.load(f)
        testX2 = pickle.load(f)
        testY  = pickle.load(f)
        validX = pickle.load(f)
        validY = pickle.load(f)
        trainX = pickle.load(f)
        trainY = pickle.load(f)
        return testX1, testX2, testY, validX, validY, trainX, trainY


testX1, testX2, testY, validX, validY, trainX, trainY = load_data()
class_num = np.max(trainY) + 1
tf.compat.v1.reset_default_graph();


In [59]:
def weight_variable(shape):
    with tf.name_scope('weights'):
        return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

In [60]:
def bias_variable(shape):
    with tf.name_scope('biases'):
        return tf.Variable(tf.zeros(shape))

In [61]:
def Wx_plus_b(weights, x, biases):
    with tf.name_scope('Wx_plus_b'):
        return tf.matmul(x, weights) + biases


In [62]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        weights = weight_variable([input_dim, output_dim])
        biases = bias_variable([output_dim])
        preactivate = Wx_plus_b(weights, input_tensor, biases)
        if act != None:
            activations = act(preactivate, name='activation')
            return activations
        else:
            return preactivate

In [63]:
def conv_pool_layer(x, w_shape, b_shape, layer_name, act=tf.nn.relu, only_conv=False):
    with tf.name_scope(layer_name):
        W = weight_variable(w_shape)
        b = bias_variable(b_shape)
        conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID', name='conv2d')
        h = conv + b
        relu = act(h, name='relu')
        if only_conv == True:
            return relu
        pool=tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID', name='max-pooling')
        return pool


In [64]:
def accuracy(y_estimate, y_real):
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
        return accuracy


In [66]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [67]:
def train_step(loss):
    with tf.name_scope('train'):

        return tf.train.AdamOptimizer(1e-4).minimize(loss)


with tf.name_scope('input'):
    h0 = tf.placeholder(tf.float32, [None, 55, 47, 3], name='x')
    y_ = tf.placeholder(tf.float32, [None, class_num], name='y')


h1 = conv_pool_layer(h0, [4, 4, 3, 20], [20], 'Conv_layer_1')

h2 = conv_pool_layer(h1, [3, 3, 20, 40], [40], 'Conv_layer_2')

h3 = conv_pool_layer(h2, [3, 3, 40, 60], [60], 'Conv_layer_3')

h4 = conv_pool_layer(h3, [2, 2, 60, 80], [80], 'Conv_layer_4', only_conv=True)


with tf.name_scope('DeepID1'):

    h3r = tf.reshape(h3, [-1, 5*4*60])

    h4r = tf.reshape(h4, [-1, 4*3*80])

    W1 = weight_variable([5*4*60, 160])
    W2 = weight_variable([4*3*80, 160])
    b = bias_variable([160])

    h = tf.matmul(h3r, W1) + tf.matmul(h4r, W2) + b

    h5 = tf.nn.relu(h)


with tf.name_scope('loss'):
    y = nn_layer(h5, 160, class_num, 'nn_layer', act=None)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = y, labels = y_))
    tf.summary.scalar('loss', loss)


accuracy = accuracy(y, y_)
optimizer = train_step(loss)
merged = tf.summary.merge_all()
saver = tf.train.Saver()

In [68]:
if __name__ == '__main__':


    def get_batch(data_x, data_y, start):
        end = (start + 1024) % data_x.shape[0]
        if start < end:
            return data_x[start:end], data_y[start:end], end
        return np.vstack([data_x[start:], data_x[:end]]), np.vstack([data_y[start:], data_y[:end]]), end

    with tf.Session() as sess:

        data_x = trainX
        data_y = (np.arange(class_num) == trainY[:,None]).astype(np.float32)
        validY = (np.arange(class_num) == validY[:,None]).astype(np.float32)


        logdir = 'log'
        if tf.gfile.Exists(logdir):
            tf.gfile.DeleteRecursively(logdir)
        tf.gfile.MakeDirs(logdir)


        sess = tf.Session()

        sess.run(tf.global_variables_initializer())

        train_writer = tf.summary.FileWriter(logdir + '/content/drive/MyDrive/train', sess.graph)
        test_writer = tf.summary.FileWriter(logdir + '/content/drive/MyDrive/test', sess.graph)


        idx = 0
        for i in range(50850):

            batch_x, batch_y, idx = get_batch(data_x, data_y, idx)
            _ = sess.run(optimizer, {h0: batch_x, y_: batch_y})

            summary = sess.run(merged, {h0: batch_x, y_: batch_y})

            train_writer.add_summary(summary, i)


            if i % 100 == 0:
                summary = sess.run(merged, {h0: validX, y_: validY})
                test_writer.add_summary(summary, i)

            if i % 5000 == 0 and i != 0:
                saver.save(sess, '/content/drive/MyDrive/checkpoint/%05d.ckpt' % i)

Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [69]:

import tensorflow as tf
from scipy.spatial.distance import cosine, euclidean

In [71]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [72]:
if __name__ == '__main__':
    with tf.Session() as sess:
        saver.restore(sess, '/content/drive/MyDrive/checkpoint/30000.ckpt')
        h1 = sess.run(h5, {h0: testX1})
        h2 = sess.run(h5, {h0: testX2})

    pre_y = np.array([cosine(x, y) for x, y in zip(h1, h2)])
    
    def part_mean(x, mask):
        z = x * mask
        return float(np.sum(z) / np.count_nonzero(z))
    
    true_mean = part_mean(pre_y, testY)
    false_mean = part_mean(pre_y, 1-testY)
    print(true_mean, false_mean)
    
    print(np.mean((pre_y < (true_mean + false_mean)/2) == testY.astype(bool)))

0.17263568522573086 0.841363694291656
0.9404761904761905
